In [1]:
import pandas as pd
import numpy as np
from scipy import spatial
import geopandas as gpd
from shapely.geometry import Point, LineString, Polygon
import fiona
import statistics
import contextily as cx
from haversine import haversine, Unit
import haversine as hs
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import matplotlib.dates as mdates

In [2]:
mypath = '../trips/'

In [3]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]

In [4]:
def filterSubstr(lists, substr):
    return [x for x in lists if substr in x]

searchString = '2022_09_14'
onlyfiles_14_09 = filterSubstr(onlyfiles, searchString)
onlyfiles_14_09

searchString = '12_48'
onlyfiles = filterSubstr(onlyfiles_14_09, searchString)
onlyfiles

['2022_09_14_LA2 Logi Jeep_12_48_12_58_2e966934-f14e-4db7-b17b-7e0c3292cd76.csv',
 '2022_09_14_PS Skylark Test_12_48_12_58_747825d8-e426-4c93-a211-90f8a31d746c.csv',
 '2022_09_14_RG Logi Jeep_12_48_12_58_296d121e-2d3b-4c71-a8c8-eb878ecba355.csv']

In [5]:
allfiles = []
for i in range(len(onlyfiles)):
    df = pd.read_csv(mypath + str(onlyfiles[i]), sep=';', decimal=',')
    df['Time[ms]'] = pd.to_datetime(df['Time[ms]'], unit='ms')
    df['driver'] = str(onlyfiles[i])[11:13]
    coord_ = [Point(x, y) for x, y in zip(df.Longitude, df.Latitude)]
    alldata_ = gpd.GeoDataFrame(df, geometry=coord_, crs ="EPSG:4326")
    allfiles.append(df)
alldata = pd.concat(allfiles, axis=0).reset_index(drop=True)
alldata['data'] = alldata['Time[ms]'].dt.date
alldata["data"] = pd.to_datetime(alldata["data"])
alldata

,Time[ms],Latitude,Longitude,Speed[m/s],Distance,Altitude[m],Flag,Acceleration[m/s^2],AccWork[J],AeroWork[J],...,TotalWork[J],StandStillTime[s],Fuel[l],CO2[kg],AccECE[%],AeroECE[%],STSECE[%],WorkECE[%],driver,data
0,2022-09-14 12:48:18,53.560533,9.980495,6.53,NaN,60.9,0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LA,2022-09-14
1,2022-09-14 12:48:19,53.560570,9.980413,6.98,6.98,60.8,0,0.45,3966.0,163.0,...,4188.795,0.0,0.000469,0.001250,242.793065,37.698433,0.000000,16.785043,LA,2022-09-14
2,2022-09-14 12:48:20,53.560601,9.980331,6.91,6.91,60.9,0,-0.07,-635.0,158.0,...,2129.205,0.0,0.000238,0.000635,122.173607,37.547440,0.000000,18.187011,LA,2022-09-14
3,2022-09-14 12:48:21,53.560636,9.980252,7.13,7.13,60.9,0,0.22,2015.0,173.0,...,3557.000,0.0,0.000398,0.001061,121.459789,37.835197,0.000000,26.629678,LA,2022-09-14
4,2022-09-14 12:48:22,53.560672,9.980171,7.01,7.01,60.9,0,-0.12,-1108.0,165.0,...,403.000,0.0,0.000045,0.000120,91.081078,37.833989,0.000000,26.141449,LA,2022-09-14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1371,2022-09-14 12:58:12,53.549796,9.967336,1.61,1.61,61.1,0,-1.30,-3835.0,2.0,...,0.000,0.0,0.000000,0.000000,133.547197,93.460074,58.019560,134.128294,RG,2022-09-14
1372,2022-09-14 12:58:13,53.549805,9.967338,0.69,0.69,61.1,0,-0.92,-1381.0,0.0,...,0.000,0.0,0.000000,0.000000,134.547659,93.596481,58.019560,134.930129,RG,2022-09-14
1373,2022-09-14 12:58:14,53.549807,9.967339,0.00,0.00,61.1,4,-0.69,-311.0,0.0,...,372.000,1.0,0.000042,0.000111,132.872395,93.727594,59.074513,133.488053,RG,2022-09-14
1374,2022-09-14 12:58:15,53.549806,9.967344,0.00,0.00,61.1,4,0.00,0.0,0.0,...,372.000,1.0,0.000042,0.000111,133.537355,93.845721,60.129465,133.588363,RG,2022-09-14


In [6]:
alldata_SORTED = alldata.sort_values(by = 'Time[ms]').reset_index(drop=True)
alldata_SORTED_cut = alldata_SORTED.dropna().reset_index(drop=True)
alldata_SORTED_cut['Time'] = pd.to_datetime(alldata_SORTED_cut['Time[ms]'].dt.strftime('%H:%M:%S'))
alldata_SORTED_cut

,Time[ms],Latitude,Longitude,Speed[m/s],Distance,Altitude[m],Flag,Acceleration[m/s^2],AccWork[J],AeroWork[J],...,StandStillTime[s],Fuel[l],CO2[kg],AccECE[%],AeroECE[%],STSECE[%],WorkECE[%],driver,data,Time
0,2022-09-14 12:48:19,53.560570,9.980413,6.98,6.98,60.8,0,0.45,3966.0,163.0,...,0.0,0.000469,0.001250,242.793065,37.698433,0.000000,16.785043,LA,2022-09-14,2023-08-05 12:48:19
1,2022-09-14 12:48:20,53.560601,9.980331,6.91,6.91,60.9,0,-0.07,-635.0,158.0,...,0.0,0.000238,0.000635,122.173607,37.547440,0.000000,18.187011,LA,2022-09-14,2023-08-05 12:48:20
2,2022-09-14 12:48:21,53.560636,9.980252,7.13,7.13,60.9,0,0.22,2015.0,173.0,...,0.0,0.000398,0.001061,121.459789,37.835197,0.000000,26.629678,LA,2022-09-14,2023-08-05 12:48:21
3,2022-09-14 12:48:22,53.560672,9.980171,7.01,7.01,60.9,0,-0.12,-1108.0,165.0,...,0.0,0.000045,0.000120,91.081078,37.833989,0.000000,26.141449,LA,2022-09-14,2023-08-05 12:48:22
4,2022-09-14 12:48:22,53.559962,9.980755,2.34,2.34,59.3,0,0.87,2162.0,6.0,...,0.0,0.000580,0.001545,300.000000,6.000643,0.000000,8.877412,RG,2022-09-14,2023-08-05 12:48:22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,2022-09-14 12:58:12,53.549796,9.967336,1.61,1.61,61.1,0,-1.30,-3835.0,2.0,...,0.0,0.000000,0.000000,133.547197,93.460074,58.019560,134.128294,RG,2022-09-14,2023-08-05 12:58:12
1368,2022-09-14 12:58:13,53.549805,9.967338,0.69,0.69,61.1,0,-0.92,-1381.0,0.0,...,0.0,0.000000,0.000000,134.547659,93.596481,58.019560,134.930129,RG,2022-09-14,2023-08-05 12:58:13
1369,2022-09-14 12:58:14,53.549807,9.967339,0.00,0.00,61.1,4,-0.69,-311.0,0.0,...,1.0,0.000042,0.000111,132.872395,93.727594,59.074513,133.488053,RG,2022-09-14,2023-08-05 12:58:14
1370,2022-09-14 12:58:15,53.549806,9.967344,0.00,0.00,61.1,4,0.00,0.0,0.0,...,1.0,0.000042,0.000111,133.537355,93.845721,60.129465,133.588363,RG,2022-09-14,2023-08-05 12:58:15


In [7]:
names = list(alldata.driver.unique())
drivers = []
for n in names:
    drivers.append(alldata_SORTED_cut[alldata_SORTED_cut.driver == n].reset_index(drop=True).drop_duplicates())
names

['LA', 'PS', 'RG']

In [8]:
# INFRASTRUCTURE

In [9]:
infr = pd.read_csv('../infrast_data_update_.csv', sep=';')
feat = list(infr.what.unique())
infr['what'].value_counts()
feat

['Turn_left',
 '4-ways intersection',
 'Traffic_light',
 'Curve',
 'Road_immission',
 'Turn_right']

In [10]:
def f(row,lat,lon):
    return hs.haversine((row['Latitude'],row['Longitude']),(lat,lon),unit=Unit.METERS)

for i in range(infr.shape[0]):
    k = infr.loc[i,'what']
    for df in drivers:
        df[k] = False

for i in range(infr.shape[0]):
    k = infr.loc[i,'what']
    coord = (infr.loc[i,'lat'],infr.loc[i,'lon'])
    for df in drivers:
        df['tmp'] = df.apply(f,axis='columns',args=coord)
        if(min(df['tmp']) < 10):
            df[k] = np.where(df['tmp'] == min(df['tmp']), True, df[k])

In [11]:
# END INFRASTRUCTURE

In [12]:
platoon_all = alldata_SORTED_cut
def f(row):
    return hs.haversine((row['Latitude_x'],row['Longitude_x']),(row['Latitude_y'],row['Longitude_y']),unit=Unit.METERS)

distances = {}

for i in range(len(drivers)):
    for j in range(i+1,len(drivers)):
        x = drivers[i][['Time','Latitude','Longitude']].merge(drivers[j][['Time','Latitude','Longitude']],on='Time')
        x['dist'] = x.apply(f,axis='columns')
        distances[names[i]+'-'+names[j]] = x

list(distances.keys())

['LA-PS', 'LA-RG', 'PS-RG']

In [13]:
platoon_all

,Time[ms],Latitude,Longitude,Speed[m/s],Distance,Altitude[m],Flag,Acceleration[m/s^2],AccWork[J],AeroWork[J],...,StandStillTime[s],Fuel[l],CO2[kg],AccECE[%],AeroECE[%],STSECE[%],WorkECE[%],driver,data,Time
0,2022-09-14 12:48:19,53.560570,9.980413,6.98,6.98,60.8,0,0.45,3966.0,163.0,...,0.0,0.000469,0.001250,242.793065,37.698433,0.000000,16.785043,LA,2022-09-14,2023-08-05 12:48:19
1,2022-09-14 12:48:20,53.560601,9.980331,6.91,6.91,60.9,0,-0.07,-635.0,158.0,...,0.0,0.000238,0.000635,122.173607,37.547440,0.000000,18.187011,LA,2022-09-14,2023-08-05 12:48:20
2,2022-09-14 12:48:21,53.560636,9.980252,7.13,7.13,60.9,0,0.22,2015.0,173.0,...,0.0,0.000398,0.001061,121.459789,37.835197,0.000000,26.629678,LA,2022-09-14,2023-08-05 12:48:21
3,2022-09-14 12:48:22,53.560672,9.980171,7.01,7.01,60.9,0,-0.12,-1108.0,165.0,...,0.0,0.000045,0.000120,91.081078,37.833989,0.000000,26.141449,LA,2022-09-14,2023-08-05 12:48:22
4,2022-09-14 12:48:22,53.559962,9.980755,2.34,2.34,59.3,0,0.87,2162.0,6.0,...,0.0,0.000580,0.001545,300.000000,6.000643,0.000000,8.877412,RG,2022-09-14,2023-08-05 12:48:22
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1367,2022-09-14 12:58:12,53.549796,9.967336,1.61,1.61,61.1,0,-1.30,-3835.0,2.0,...,0.0,0.000000,0.000000,133.547197,93.460074,58.019560,134.128294,RG,2022-09-14,2023-08-05 12:58:12
1368,2022-09-14 12:58:13,53.549805,9.967338,0.69,0.69,61.1,0,-0.92,-1381.0,0.0,...,0.0,0.000000,0.000000,134.547659,93.596481,58.019560,134.930129,RG,2022-09-14,2023-08-05 12:58:13
1369,2022-09-14 12:58:14,53.549807,9.967339,0.00,0.00,61.1,4,-0.69,-311.0,0.0,...,1.0,0.000042,0.000111,132.872395,93.727594,59.074513,133.488053,RG,2022-09-14,2023-08-05 12:58:14
1370,2022-09-14 12:58:15,53.549806,9.967344,0.00,0.00,61.1,4,0.00,0.0,0.0,...,1.0,0.000042,0.000111,133.537355,93.845721,60.129465,133.588363,RG,2022-09-14,2023-08-05 12:58:15


In [14]:
plt.figure(figsize = (20, 8))
plt.rcParams.update({'font.size': 22})
# plot feat
ref_driver_4_infr = drivers[0]
for i in range(len(feat)):
    first = True
    f = feat[i]
    c = list(mcolors.BASE_COLORS.values())[i]
    for t in platoon_all[ref_driver_4_infr[f]]['Time']:
        if first:
            plt.axvline(t,color=c,label=f)
            first = False
        else:
             plt.axvline(t,color=c)

keys = list(distances.keys())
                
for k in keys:
    plt.plot(distances[k]['Time'],distances[k]['dist'],label=k,linewidth=4)
plt.gcf().axes[0].xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
plt.xlabel('Time')
plt.ylabel('Distance (m)')
plt.legend(loc='best')
#plt.savefig('images/dist.pdf',bbox_inches='tight')
plt.show()

C:\Users\marco\AppData\Local\Temp\ipykernel_11696\1530049298.py:9: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  for t in platoon_all[ref_driver_4_infr[f]]['Time']:


IndexingError: Unalignable boolean Series provided as indexer (index of the boolean Series and of the indexed object do not match).

<Figure size 2000x800 with 0 Axes>

In [ ]:
dist_v = pd.DataFrame(platoon_alldist, columns = ['RG_LA', 'LA_PS'])
pd.DataFrame(platoon_alldist, columns = ['RG_LA', 'LA_PS']).to_csv('distance_platoon.csv', index=False)

In [ ]:
dist_coord = pd.concat([pd.DataFrame(platoon_all_lat_dist, columns=['v1_lat', 'v2_lat'])['v1_lat'], pd.DataFrame(platoon_all_lon_dist, columns=['v1_lon', 'v2_lon'])['v1_lon'],pd.DataFrame(platoon_all_lat_dist, columns=['v1_lat', 'v2_lat'])['v2_lat'], pd.DataFrame(platoon_all_lon_dist, columns=['v1_lon', 'v2_lon'])['v2_lon']], axis=1)

In [ ]:
pd.concat([pd.DataFrame(platoon_all_lat_dist, columns=['v1_lat', 'v2_lat'])['v1_lat'], pd.DataFrame(platoon_all_lon_dist, columns=['v1_lon', 'v2_lon'])['v1_lon'],pd.DataFrame(platoon_all_lat_dist, columns=['v1_lat', 'v2_lat'])['v2_lat'], pd.DataFrame(platoon_all_lon_dist, columns=['v1_lon', 'v2_lon'])['v2_lon']], axis=1).to_csv('platoon_distances_coord.csv', index=False)

In [ ]:
infr = pd.read_csv('../infrast_data_update_.csv', sep=';')
infr.what.unique()

In [ ]:
drivers[0]

In [ ]:
allveic = []
for k in range(0, len(dist_coord.columns)-1, 2):
    veic = []
    for i in range(len(dist_coord)):
        infra = []
        for j in range(len(infr)):
            inf = (infr.iloc[j].lon,infr.iloc[j].lat)
            distcoor = (dist_coord.iloc[i][str(dist_coord.columns[k+1])], dist_coord.iloc[i][str(dist_coord.columns[k])])
            infra.append(hs.haversine(inf,distcoor,unit=Unit.METERS))
        veic.append(infra)
    allveic.append(veic)

In [ ]:
v1 = pd.DataFrame(allveic[0])
v2 = pd.DataFrame(allveic[1])

In [ ]:
v1

In [ ]:
allfeat = []
for i in range(len(v1)): 
    allfeat.append(list(v1[v1<500].iloc[i].dropna()))
new_v1 = pd.DataFrame(allfeat).iloc[:,0:7].fillna(0)

In [ ]:
allfeat = []
for i in range(len(v2)): 
    allfeat.append(list(v2[v2<500].iloc[i].dropna()))
new_v2 = pd.DataFrame(allfeat).iloc[:,0:7].fillna(0)

In [ ]:
v1.columns = infr.what
v2.columns = infr.what

In [ ]:
vlist = []
vlist.append(v1)
vlist.append(v2)

In [ ]:
import statsmodels.api as sm

In [ ]:
all_X = []
all_y = []

for k in range(len(vlist)):
    feats = []
    v = vlist[k]
    infrast = v.idxmin(axis=1)
    for i in range(len(v)):
        if (v.iloc[i][str(infrast[i])] < 80).any():
            feats.append([infrast[i]])
        else:
            feats.append([])
    new_v = pd.get_dummies(pd.DataFrame(feats), prefix='Feat')

    X = new_v.copy()
    y = dist_v[str(dist_v.columns[k])]
    X['old_dist'] = dist_v[str(dist_v.columns[k])].shift(10).fillna(0)
    all_X.append(X)
    all_y.append(y)

all_X_ = pd.concat(all_X).reset_index(drop=True).fillna(0)
all_y_ = pd.concat(all_y).reset_index(drop=True).fillna(0)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

In [ ]:
results_summary = model.summary()
df = pd.DataFrame.from_records(results_summary.tables[1].data)
header = df.iloc[0]
df = df[1:] # take the data less the header row
df.columns = header
df.to_csv('platoon_II_14_09_12_48_.csv')

In [ ]:
platoon_all_CO2_ = pd.DataFrame(platoon_all_CO2, columns = ['RG_LA', 'LA_PS'])
pd.DataFrame(platoon_all_CO2, columns = ['RW_LA', 'LA_PS']).to_csv('CO2_platoon.csv', index=False)

all_X = []
all_y = []

for k in range(len(vlist)):
    feats = []
    v = vlist[k]
    infrast = v.idxmin(axis=1)
    for i in range(len(v)):
        if (v.iloc[i][str(infrast[i])] <80).any():
            feats.append([infrast[i]])
        else:
            feats.append([])
    new_v = pd.get_dummies(pd.DataFrame(feats), prefix='Feat')

    X = new_v.copy()
    y = platoon_all_CO2_[str(platoon_all_CO2_.columns[k])]
    X['old_dist'] = dist_v[str(dist_v.columns[k])].shift(10).fillna(0)
    all_X.append(X)
    all_y.append(y)

all_X_ = pd.concat(all_X).reset_index(drop=True).fillna(0)
all_y_ = pd.concat(all_y).reset_index(drop=True).fillna(0)
model = sm.OLS(y, X).fit()
predictions = model.predict(X)
model.summary()

In [ ]:
results_summary = model.summary()
df = pd.DataFrame.from_records(results_summary.tables[1].data)
header = df.iloc[0]
df = df[1:] # take the data less the header row
df.columns = header
df.to_csv('platoon_II_14_09_12_48_CO2.csv')